In [39]:
import pandas as pd
import sklearn
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LassoCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np
%matplotlib inline

import statsmodels.api as sm
from statsmodels.api import OLS

In [40]:
crime_property_light_population= pd.read_csv("./model_data/crime_light_density.csv")
crime_property_light_population.head()

resp_predictors = ['OFFENSE_CODE_GROUP','SHOOTING', 'MONTH', 'DAY_OF_WEEK', 'HOUR', 'Population density (per square mile of land area)', 'BLDG_VAL', 'LAND_VAL','light_density']
df = crime_property_light_population[resp_predictors]
df = df.dropna()

In [41]:
#Let's label encode & one-hot encode the Categoricals (SHOOTING, DAY_OF_WEEK)

enc = OneHotEncoder(handle_unknown='ignore')

le = LabelEncoder() 
  
df['SHOOTING']= le.fit_transform(df['SHOOTING'])
df['DAY_OF_WEEK']= le.fit_transform(df['DAY_OF_WEEK']) 
df['OFFENSE_CODE_GROUP'] = le.fit_transform(df['OFFENSE_CODE_GROUP'])

In [42]:
df_cat = enc.fit_transform(df[["MONTH","SHOOTING","DAY_OF_WEEK"]]).toarray()

df_cat = pd.DataFrame(df_cat, columns=enc.get_feature_names(['MONTH','SHOOTING', 'DAY_OF_WEEK']))
df = pd.concat([df, df_cat], axis=1)
df = df.dropna()

df.head()

,OFFENSE_CODE_GROUP,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_1,...,MONTH_12,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
0,4.0,0.0,10.0,3.0,9.0,6206.9,455.644447,21.073507,0.437849,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,4.0,0.0,10.0,2.0,20.0,6206.9,455.644447,21.073507,0.437849,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.0,0.0,10.0,2.0,15.0,16212.0,2250.588503,63.554783,0.582893,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4.0,0.0,10.0,2.0,4.0,32724.1,14799.401260,480.463801,0.656585,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,4.0,0.0,10.0,6.0,8.0,12192.0,278.377510,165.249738,0.678941,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:

X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'OFFENSE_CODE_GROUP'], 
                                                         df.OFFENSE_CODE_GROUP, test_size=0.2, 
                                                         random_state = 109, 
                                                         stratify = df.OFFENSE_CODE_GROUP)

In [45]:
y_train = np.array(y_train)
y_test = np.array(y_test)

display(X_train.head())
print(type(y_train))
print(y_train)

,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_1,MONTH_2,...,MONTH_12,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
258928,0.0,12.0,0.0,13.0,12192.0,278.377510,165.249738,0.678941,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
319203,0.0,4.0,3.0,15.0,15913.2,1293.230035,31.015855,0.568578,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
84862,0.0,10.0,4.0,12.0,14884.4,1040.984096,31.260815,0.640733,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
372291,0.0,9.0,5.0,15.0,5669.8,97.055959,27.951491,0.486357,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
71941,0.0,10.0,1.0,0.0,40159.2,8464.863983,333.361435,0.714606,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


<class 'numpy.ndarray'>
[40. 33. 29. ... 55. 33. 29.]


In [46]:
#Let's scale the variables
min_max_scaler = MinMaxScaler()
numerical = ["HOUR", "BLDG_VAL", "LAND_VAL", "light_density", "MONTH", "Population density (per square mile of land area)" ]
X_normalized = (X_train - X_train.min(axis=0)) / (X_train.max(axis=0) - X_train.min(axis=0))
min_max_scaler.fit(X_train)
X_normalized_tst = pd.DataFrame(min_max_scaler.transform(X_test))
X_normalized_tst.columns = X_test.columns


display(X_normalized.head())
display(X_normalized_tst.head())


,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_1,MONTH_2,...,MONTH_12,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
258928,0.0,1.000000,0.000000,0.565217,0.149621,0.001422,0.190190,0.558139,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
319203,0.0,0.272727,0.500000,0.652174,0.205656,0.009384,0.013115,0.406251,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
84862,0.0,0.818182,0.666667,0.521739,0.190164,0.007405,0.013439,0.505555,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
372291,0.0,0.727273,0.833333,0.652174,0.051409,0.000000,0.009073,0.293094,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
71941,0.0,0.818182,0.166667,0.000000,0.570757,0.065643,0.411953,0.607225,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


,SHOOTING,MONTH,DAY_OF_WEEK,HOUR,Population density (per square mile of land area),BLDG_VAL,LAND_VAL,light_density,MONTH_1,MONTH_2,...,MONTH_12,SHOOTING_0,SHOOTING_1,DAY_OF_WEEK_0,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6
0,0.0,0.272727,0.333333,0.695652,0.059496,0.002813,0.000000,0.226334,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.000000,0.333333,0.434783,0.210523,0.008147,0.033911,0.414481,0.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.090909,0.000000,0.565217,0.140010,0.000189,0.017472,0.378771,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.818182,0.166667,0.695652,0.139273,0.001371,0.010112,0.352743,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.545455,0.500000,0.695652,0.205656,0.009384,0.013115,0.406251,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# Let's first start with Logistic Model

logreg = LogisticRegression(random_state = 0, solver='lbfgs', multi_class='multinomial', max_iter = 1000)
logreg.fit(X_normalized, y_train)
y_pred = logreg.predict(X_normalized_tst)

In [56]:
mcm = multilabel_confusion_matrix(y_test, y_pred, sample_weight=None, labels=None, samplewise=False)
# logreg.score(X_normalized, y_test)
# print(mcm)

In [57]:
print(y_pred[:10])
print(y_test[:10])
count = 0
for i, j in zip(y_pred, y_test):
    if i == j:
        count += 1

accuracy = count / len(y_pred)
print(accuracy)
    

[42. 42. 42. 42. 42. 33. 42. 42. 42. 42.]
[63. 42. 65. 42.  4. 33. 63. 19. 60. 62.]
0.12690523087498512


In [ ]:
logreg.score(X_normalized_tst, y_test)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_normalized, y_train) 

In [58]:
  
# accuracy on X_test 
accuracy = knn.score(X_normalized_tst, y_test)

In [59]:
print(accuracy)

0.09036709981889567


In [50]:
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)

print(cm)

[[155   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  1   0   0 ...   0   0   0]
 [ 17   0   0 ...   0   0   0]]


In [51]:
import pickle

pickle.dump(logreg, open('logreg.sav', 'wb'))
pickle.dump(knn, open('knn.sav', 'wb'))